In [1]:
import importlib
import JeffUtils
importlib.reload(JeffUtils)

<module 'JeffUtils' from 'C:\\Users\\j3ff1\\Documents\\School Work\\MLProjects\\TelecomChurnAnalysis\\JeffUtils.py'>

data from: https://www.kaggle.com/datasets/abhinav89/telecom-customer?resource=download

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import tqdm 
import matplotlib.pyplot as plt
import math
import scipy
import re

In [2]:
pd.set_option('display.max_columns', None)

In [33]:
RANDOM_SEED = 1337

# Loading Raw Data

In [3]:
raw_data = pd.read_csv('telecomChurn.zip')

In [4]:
raw_data.head(5)

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,change_rev,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,comp_vce_Mean,comp_dat_Mean,custcare_Mean,ccrndmou_Mean,cc_mou_Mean,inonemin_Mean,threeway_Mean,mou_cvce_Mean,mou_cdat_Mean,mou_rvce_Mean,owylis_vce_Mean,mouowylisv_Mean,iwylis_vce_Mean,mouiwylisv_Mean,peak_vce_Mean,peak_dat_Mean,mou_peav_Mean,mou_pead_Mean,opk_vce_Mean,opk_dat_Mean,mou_opkv_Mean,mou_opkd_Mean,drop_blk_Mean,attempt_Mean,complete_Mean,callfwdv_Mean,callwait_Mean,churn,months,uniqsubs,actvsubs,new_cell,crclscod,asl_flag,totcalls,totmou,totrev,adjrev,adjmou,adjqty,avgrev,avgmou,avgqty,avg3mou,avg3qty,avg3rev,avg6mou,avg6qty,avg6rev,prizm_social_one,area,dualband,refurb_new,hnd_price,phones,models,hnd_webcap,truck,rv,ownrent,lor,dwlltype,marital,adults,infobase,income,numbcars,HHstatin,dwllsize,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,23.9975,219.25,22.500,0.2475,0.00,0.0,0.0,0.0,0.0,-157.25,-18.9975,0.666667,0.0,0.666667,0.0,6.333333,0.0,52.333333,0.0,42.333333,0.0,45.000000,0.0,0.000000,0.000000,0.000000,18.000000,0.000000,90.643333,0.0,97.176667,0.000000,0.000000,0.000000,0.000000,58.000000,0.0,132.600000,0.0,24.000000,0.0,55.220000,0.0,1.333333,52.333333,45.000000,0.0,0.333333,1,61,2,1,U,A,N,1652,4228.00000,1504.62,1453.44,4085.00,1602,29.66,83.37,32.69,272,116,30,322.0,136.0,38.0,S,NORTHWEST/ROCKY MOUNTAIN AREA,Y,N,149.98999,2.0,2.0,WCMB,0.0,0.0,O,15.0,S,S,1.0,M,4.0,3.0,C,A,0.0,N,U,U,U,U,U,Y,361.0,1000001
1,57.4925,482.75,37.425,0.2475,22.75,9.1,9.1,0.0,0.0,532.25,50.9875,8.333333,0.0,1.000000,0.0,61.333333,0.0,263.333333,0.0,69.000000,0.0,193.333333,0.0,1.666667,6.333333,5.463333,53.000000,0.333333,189.396667,0.0,55.280000,46.333333,24.216667,6.333333,3.696667,83.666667,0.0,75.333333,0.0,157.000000,0.0,169.343333,0.0,9.333333,263.333333,193.333333,0.0,5.666667,0,56,1,1,N,EA,N,14654,26400.00000,2851.68,2833.88,26367.00,14624,51.53,479.40,265.89,305,158,40,477.0,275.0,48.0,U,CHICAGO AREA,N,N,NaN,7.0,6.0,WC,1.0,1.0,NaN,1.0,S,S,1.0,M,5.0,1.0,C,A,0.0,Z,U,U,U,U,U,Y,240.0,1000002
2,16.9900,10.25,16.990,0.0000,0.00,0.0,0.0,0.0,0.0,-4.25,0.0000,0.333333,0.0,0.000000,0.0,2.666667,0.0,9.000000,0.0,0.333333,0.0,6.000000,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,5.426667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.0,5.193333,0.0,1.000000,0.0,0.233333,0.0,0.333333,9.000000,6.000000,0.0,0.000000,1,58,1,1,Y,C,N,7903,24385.05333,2155.91,1934.47,24303.05,7888,34.54,433.98,140.86,12,7,17,11.0,6.0,17.0,S,GREAT LAKES AREA,N,N,29.98999,2.0,1.0,NaN,0.0,0.0,O,7.0,S,M,2.0,M,5.0,2.0,C,A,0.0,N,U,Y,U,U,U,Y,1504.0,1000003
3,38.0000,7.50,38.000,0.0000,0.00,0.0,0.0,0.0,0.0,-1.50,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,3.666667,0.0,1.333333,0.0,3.666667,0.0,0.000000,0.000000,0.000000,1.333333,0.000000,8.410000,0.0,0.413333,0.333333,0.256667,0.000000,0.000000,1.333333,0.0,3.380000,0.0,3.666667,0.0,5.450000,0.0,0.000000,3.666667,3.666667,0.0,0.000000,0,60,1,1,Y,B,N,1502,3065.00000,2000.90,1941.81,3035.00,1479,40.45,63.23,30.81,8,3,38,50.0,25.0,40.0,T,CHICAGO AREA,N,N,29.98999,1.0,1.0,NaN,0.0,0.0,NaN,6.0,M,M,4.0,M,6.0,1.0,C,D,0.0,U,Y,U,U,U,U,Y,1812.0,1000004
4,55.2300,570.50,71.980,0.0000,0.00,0.0,0.0,0.0,0.0,38.50,0.0000,9.666667,0.0,0.666667,0.0,77.000000,0.0,222.333333,0.0,94.666667,0.0,137.000000,0.0,8.666667,15.000000,11.076667,66.000000,0.000000,285.233333,0.0,106.330000,14.666667,10.816667,0.666667,0.366667,97.333333,0.0,173.476667,0.0,90.333333,0.0,218.086667,0.0,10.333333,222.333333,137.000000,0.0,0.000000,0,57,1,1,Y,A,N,4485,14028.00000,2181.12,2166.48,13965.00,4452,38.69,249.38,79.50,558,191,55,586.0,196.0,80.0,U,NEW ENGLAND AREA,Y,N,149.98999,6.0,4.0,WCMB,0.0,0.0,R,5.0,M,S,1.0,M,6.0,1.0,C,O,0.0,I,U,U,U,U,U,Y,434.0,1000005


## Handling column descriptions

Since there is literally a hundred columns in this dataset, lets take a general look at what kind of data is available. 

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import spacy

### read-in column descriptions

In [6]:
col_desc = pd.read_json('columnDescriptions.json',orient='index')[0].rename('col_desc').to_frame()

In [7]:
col_desc.head(5)

,col_desc
rev_Mean,Mean monthly revenue (charge amount)
mou_Mean,Mean number of monthly minutes of use
totmrc_Mean,Mean total monthly recurring charge
da_Mean,Mean number of directory assisted calls
ovrmou_Mean,Mean overage minutes of use


### Preprocess column descriptions

In [8]:
#Remove stop words
col_desc['clean_desc'] = (
    col_desc['col_desc']
    .apply(gensim.parsing.preprocessing.remove_stopwords)
)
#Remove characters that are not decimals,
#not letters, and not white space
col_desc['clean_desc'] = (
    col_desc['clean_desc']
    .str.replace('[^a-zA-Z0-9_ ]','')
)
#Replace double whitespace with single space
#created from operation above
col_desc['clean_desc'] = (
    col_desc['clean_desc']
    .str.replace('  ',' ')
)

In [9]:
nlp = spacy.load('en_core_web_sm')

In [10]:
#Tokenize words in the cleaned description and lemmatize them
col_desc['tokenized'] = col_desc['clean_desc'].apply(lambda x: [token.lemma_ for token in nlp(x)])
#Join lemmatized tokens back into single string for ease of use
col_desc['lemmatized'] = col_desc['tokenized'].apply(lambda x: ' '.join(x))

In [11]:
col_desc

,col_desc,clean_desc,tokenized,lemmatized
rev_Mean,Mean monthly revenue (charge amount),Mean monthly revenue charge amount,"[mean, monthly, revenue, charge, amount]",mean monthly revenue charge amount
mou_Mean,Mean number of monthly minutes of use,Mean number monthly minutes use,"[mean, number, monthly, minute, use]",mean number monthly minute use
totmrc_Mean,Mean total monthly recurring charge,Mean total monthly recurring charge,"[mean, total, monthly, recur, charge]",mean total monthly recur charge
da_Mean,Mean number of directory assisted calls,Mean number directory assisted calls,"[mean, number, directory, assist, call]",mean number directory assist call
ovrmou_Mean,Mean overage minutes of use,Mean overage minutes use,"[mean, overage, minute, use]",mean overage minute use
...,...,...,...,...
kid11_15,Child 11 - 15 years of age in household,Child 11 15 years age household,"[child, 11, 15, year, age, household]",child 11 15 year age household
kid16_17,Child 16 - 17 years of age in household,Child 16 17 years age household,"[child, 16, 17, year, age, household]",child 16 17 year age household
creditcd,Credit card indicator,Credit card indicator,"[credit, card, indicator]",credit card indicator
eqpdays,Number of days (age) of current equipment,Number days age current equipment,"[number, day, age, current, equipment]",number day age current equipment


### Bag of words

In [12]:
tokenizedDesc = col_desc['tokenized'].to_list()
dictionary = gensim.corpora.Dictionary(tokenizedDesc)
corpus = [dictionary.doc2bow(text) for text in tokenizedDesc]

### LDA grouping

In [15]:
from gensim.models import CoherenceModel

In [16]:
ldas = []
for n in range(3,16):
    lda = gensim.models.ldamodel.LdaModel(
        corpus, num_topics=n,id2word=dictionary, random_state=RANDOM_SEED
    )
    ldas.append((n,lda))

In [244]:
coher_scores = []
for lda_tup in ldas:
    coher_model = CoherenceModel(model=lda_tup[1], texts=tokenizedDesc, dictionary=dictionary, coherence='c_v')
    coher_scores.append((lda_tup[0],coher_model.get_coherence()))

In [245]:
coher_scores

[(3, 0.5067327030933383),
 (4, 0.5228109088095966),
 (5, 0.4975643925681844),
 (6, 0.484679537033542),
 (7, 0.4777373076042672),
 (8, 0.4800708640499987),
 (9, 0.47900234265635877),
 (10, 0.49272810795529887),
 (11, 0.4838953474770993),
 (12, 0.5076580159522219),
 (13, 0.4988016907739328),
 (14, 0.5153082055690436),
 (15, 0.5053040277667462)]

In [264]:
lda = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=4,id2word=dictionary, random_state=RANDOM_SEED
)

In [265]:
#get table of probabilities for each topic
topics_probs = np.array(lda.get_document_topics(corpus))[:,:,1]

In [269]:
col_desc[col_desc.col_desc.str.contains('household')]

,col_desc,clean_desc,tokenized,lemmatized,topic_ids
uniqsubs,Number of unique subscribers in the household,Number unique subscribers household,"[number, unique, subscriber, household]",number unique subscriber household,13
actvsubs,Number of active subscribers in household,Number active subscribers household,"[number, active, subscriber, household]",number active subscriber household,3
adults,Number of adults in household,Number adults household,"[number, adult, household]",number adult household,11
HHstatin,Premier household status indicator,Premier household status indicator,"[Premier, household, status, indicator]",Premier household status indicator,6
kid0_2,Child 0 - 2 years of age in household,Child 0 2 years age household,"[child, 0, 2, year, age, household]",child 0 2 year age household,2
kid3_5,Child 3 - 5 years of age in household,Child 3 5 years age household,"[child, 3, 5, year, age, household]",child 3 5 year age household,14
kid6_10,Child 6 - 10 years of age in household,Child 6 10 years age household,"[child, 6, 10, year, age, household]",child 6 10 year age household,2
kid11_15,Child 11 - 15 years of age in household,Child 11 15 years age household,"[child, 11, 15, year, age, household]",child 11 15 year age household,9
kid16_17,Child 16 - 17 years of age in household,Child 16 17 years age household,"[child, 16, 17, year, age, household]",child 16 17 year age household,14


In [266]:
topics_probs

array([[0.04209142, 0.86476809, 0.05011833, 0.04302216],
       [0.04429376, 0.04535958, 0.43783519, 0.47251147],
       [0.04237673, 0.044152  , 0.38022074, 0.53325051],
       [0.04316218, 0.87169874, 0.04214286, 0.04299623],
       [0.05065949, 0.31103835, 0.05305242, 0.58524978],
       [0.06349503, 0.80685657, 0.06574827, 0.06390016],
       [0.05177119, 0.84408081, 0.05175211, 0.05239588],
       [0.05058266, 0.84609461, 0.05174812, 0.05157458],
       [0.05232098, 0.84490412, 0.05071569, 0.05205928],
       [0.02501404, 0.02505334, 0.92431378, 0.02561887],
       [0.02779425, 0.02798015, 0.91642338, 0.02780216],
       [0.03751224, 0.88906634, 0.03604035, 0.03738106],
       [0.03685168, 0.89011019, 0.03603939, 0.03699872],
       [0.03757391, 0.88900423, 0.03604051, 0.03738138],
       [0.03687564, 0.89008695, 0.03603949, 0.03699793],
       [0.8678956 , 0.044847  , 0.04221397, 0.04504346],
       [0.37689289, 0.5350585 , 0.04219442, 0.04585423],
       [0.03734506, 0.88923651,

The topics_probs table looks akin to below: 
\begin{bmatrix} text_1\ topic_1\ probability & ... & text_1\ topic_n\ probability \\ \vdots & \ddots & \vdots \\ text_n\ topic_1\ probability & ... & text_n\ topic_n\ probability \end{bmatrix}

Where each text is the description of our column name, topic being a topic found by our LDA model, and probability being the probability that the text belongs to the topic

In [256]:
topic_ids = (
    np.hstack([
        np.where(topics_probs>0.75,topics_probs,-1), #turn probabilities <0.5 into -1
        np.zeros((topics_probs.shape[0],1)) #add a column of 0s for outliers
    ])
    .argmax(1) #since each column represents a topic created by LDA (or column representing outliers), 
                #get column with maximum probability, to yield topic each row most likely belongs to
)
col_desc['topic_ids'] = topic_ids

In [257]:
col_desc['topic_ids'].value_counts()

14    21
8     12
1     10
13     8
11     8
10     8
12     5
7      5
5      5
0      5
6      4
2      4
4      3
9      1
3      1
Name: topic_ids, dtype: int64

In [263]:
col_desc[col_desc['topic_ids']==4]

,col_desc,clean_desc,tokenized,lemmatized,topic_ids
totmrc_Mean,Mean total monthly recurring charge,Mean total monthly recurring charge,"[mean, total, monthly, recur, charge]",mean total monthly recur charge,4
threeway_Mean,Mean number of three way calls,Mean number way calls,"[mean, number, way, call]",mean number way call,4
months,Total number of months in service,Total number months service,"[total, number, month, service]",total number month service,4
